In [304]:
from consensus.PBFT_Simulator import PBFT_Simulator
from consensus.QPBFT_Simulator import QPBFT_Simulator
from consensus.Proposed_Simulator import Proposed_Simulator
import time

In [305]:
consensus = PBFT_Simulator(10, 2)

In [306]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
delay = end-start
print(f"Spend time => {delay} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 1, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 2, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 3, Faulty => False, Send_messages => ('Hello world 0', 'pre-prepare', 3), Receive_messages => [('Hello world 0', 'pre-prepare', 3, '239ff352373082640bdb99c3b2c9fd9c2f48fd70df44216d57d619db489cc6cb')]
IdUser => 4, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 5, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 6, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 7, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 8, Faulty => Fals

In [307]:
del consensus

In [308]:
consensus = QPBFT_Simulator(6, 4, 2)

In [309]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
spend = end-start
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0,Score => 0.00, Role =>  Manager, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 0)    , Receive_messages_log => [('Hello world 0', 'prepare', 0, '5310eca6643a80e0bafcc136e6c89b61921827e21289714b66752fb1d9425448')]
IdUser => 2,Score => 0.68, Role =>  Manager, Faulty =>   True, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 3,Score => 0.81, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 8,Score => 1.00, Role =>    Voter, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
------------------------------
Prepare Phase
IdUser => 0,Score => 0.00, Role =>  Manager, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 0)    , Receive_messages_log => [('Hello world 0', 'prepare', 0, '5310eca6643a80e0bafcc136e6c89b61921827e21289714b66752fb1d9425448')]
IdUser =

In [310]:
del consensus

In [311]:
consensus = Proposed_Simulator(6, 4, 5, 3)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 6, Priority => 1.00, Role =>   Slave, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 6)    , Receive_Messages_Log => [('Hello world 0', 'prepare', 6, 'ea9b28ff49b0961bc2d8652f13862064c4cb3bda24c10a8e4965202fb6e4a671')]
IdUser => 9, Priority => 0.44, Role =>   Slave, Faulty =>  False, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 4, Priority => 0.37, Role =>  Master, Faulty =>  False, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 1, Priority => 0.19, Role =>  Master, Faulty =>   True, Send_message_log => ''                                 , Receive_Messages_Log => []
IdUser => 5, Priority => 0.09, Role =>  Master, Faulty =>   True, Send_message_log => ''                                 , Receive_Messages_Log => []
------------------------------
Prepare Phase
IdUser => 6, Priority => 1.00, Role =>   Slave, Faulty =>  False, Send_message_log => ('Hell

In [312]:
del consensus